# AI Talks Campaign Analysis (SQL + Python)

This notebook builds a SQLite database from processed CSVs, runs SQL analysis, and visualizes key insights.

In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Local project imports
try:
    from scripts.config import ROOT, FILES
    from scripts import build_db
except ModuleNotFoundError:
    import sys
    sys.path.append(str(Path.cwd() / 'scripts'))
    from config import ROOT, FILES  # type: ignore
    import build_db  # type: ignore

DB_PATH = ROOT / 'data' / 'ai_talks.sqlite'
DB_PATH

## Build the SQLite database from processed CSVs

In [ ]:
# Ensure processed CSVs exist (see scripts/extract_from_youtube.py if needed)
for name, p in FILES.items():
    print(name, Path(p).exists(), p)

# Build/refresh the SQLite database
build_db.build_sqlite(DB_PATH)
DB_PATH.exists()

## Helper to run SQL

In [ ]:
def read_sql(query: str) -> pd.DataFrame:
    with sqlite3.connect(DB_PATH) as con:
        return pd.read_sql_query(query, con)

# Quick peek at tables
read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

## Top-performing videos

In [ ]:
top_videos = read_sql("
SELECT COALESCE(video_id,'') AS video_id, COALESCE(title,'') AS title, CAST(views AS INTEGER) AS views
FROM content
ORDER BY CAST(views AS INTEGER) DESC
LIMIT 10;
")
top_videos

In [ ]:
plt.figure(figsize=(10,5))
plt.barh(top_videos['title'], top_videos['views'])
plt.gca().invert_yaxis()
plt.title('Top 10 Videos by Views')
plt.xlabel('Views')
plt.ylabel('Video')
plt.tight_layout()
plt.show()

## Traffic sources

In [ ]:
traffic = read_sql("
SELECT COALESCE(traffic_source,'Unknown') AS traffic_source, SUM(CAST(views AS INTEGER)) AS total_views
FROM traffic
GROUP BY traffic_source
ORDER BY total_views DESC;
")
traffic

In [ ]:
plt.figure(figsize=(10,5))
plt.barh(traffic['traffic_source'], traffic['total_views'])
plt.gca().invert_yaxis()
plt.title('Traffic Sources by Views')
plt.xlabel('Views')
plt.ylabel('Source')
plt.tight_layout()
plt.show()

## Top countries

In [ ]:
countries = read_sql("
SELECT COALESCE(country,'Unknown') AS country, SUM(CAST(views AS INTEGER)) AS total_views
FROM geography
GROUP BY country
ORDER BY total_views DESC
LIMIT 10;
")
countries

In [ ]:
plt.figure(figsize=(10,5))
plt.barh(countries['country'], countries['total_views'])
plt.gca().invert_yaxis()
plt.title('Top Countries by Views')
plt.xlabel('Views')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

## Subscribers gained over time

In [ ]:
subs = read_sql("
SELECT date, SUM(CAST(subs_gained AS REAL)) AS subs_gained
FROM dates
GROUP BY date
ORDER BY date ASC;
")
subs['date'] = pd.to_datetime(subs['date'], errors='coerce')
subs = subs.dropna(subset=['date'])
subs

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(subs['date'], subs['subs_gained'])
plt.title('Subscribers Gained Over Time')
plt.xlabel('Date')
plt.ylabel('Subscribers Gained')
plt.tight_layout()
plt.show()

## Optional: Generate figures and reports artifacts

In [ ]:
# Run the EDA to save figures and KPIs
from scripts import eda_youtube
dfs = eda_youtube.load_data()
eda_youtube.validate_inputs(dfs)
kpis = eda_youtube.kpi_table(dfs)
kpis


In [ ]:
# Generate data dictionary, executive summary PDF, and PPTX deck
from scripts.generate_reports import main as generate_reports_main
generate_reports_main()
print('Reports generated in /reports')